In [8]:
import numpy as np
import networkx as nx
import plotly

import node2vec

import torch
import torch_geometric as tg

from sklearn.manifold import SpectralEmbedding
from sklearn.cluster import KMeans

In [2]:
def plot_graph(G, pos):
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = plotly.graph_objects.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = plotly.graph_objects.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys', 'YlGnBu', 'Greens', 'YlOrRd', 'Bluered', 'RdBu',
            #'Reds', 'Blues', 'Picnic', 'Rainbow', 'Portland', 'Jet',
            #'Hot', 'Blackbody', 'Earth', 'Electric', 'Viridis', 'Cividis'
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=5,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(f'#{node+1} has {len(adjacencies[1])} connections')

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    fig = plotly.graph_objects.Figure(data=[edge_trace, node_trace],
            layout=plotly.graph_objects.Layout(
                title='<br>Network graph',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Graph Network",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

    fig.show()

In [3]:
G_IP = nx.Graph()

with open('data/tech-ip/tech-ip.edges', 'r') as file:
    for line in file:
        if line.startswith('%'):
            continue
        u, v = map(int, line.strip().split())
        G_IP.add_edge(u, v)

print(G_IP.number_of_nodes(), G_IP.number_of_edges())
pos = nx.spring_layout(G_IP)

In [ ]:
G_P2P = nx.Graph()

with open('data/p2p/p2p-Gnutella31.txt', 'r') as file:
    for line in file:
        if line.startswith('#'):
            continue
        u, v = map(int, line.strip().split())
        G_P2P.add_edge(u, v)

print(G_P2P.number_of_nodes(), G_P2P.number_of_edges())
pos = nx.spring_layout(G_P2P)

In [3]:
G_EURO_ROAD = nx.Graph()

with open('data/inf-euroroad/inf-euroroad.edges', 'r') as file:
    for line in file:
        if line.startswith('%'):
            continue
        u, v = map(int, line.strip().split())
        G_EURO_ROAD.add_edge(u-1, v-1)

print(G_EURO_ROAD.number_of_nodes(), G_EURO_ROAD.number_of_edges())
pos = nx.spring_layout(G_EURO_ROAD)

1174 1417


In [4]:
G = G_EURO_ROAD
plot_graph(G, pos)

In [11]:
EPOCHS = 100
LR = 0.01

In [12]:
edge_index = torch.tensor(list(G.edges)).t().contiguous()
x = torch.eye(G.number_of_nodes())
y = torch.zeros(G.number_of_nodes(), dtype=torch.long)
# y = torch.randint(0, NUM_CLASSES, (G.number_of_nodes(),))
data = tg.data.Data(x=x, edge_index=edge_index)

# generate train, val and test masks
train_mask = torch.zeros(G.number_of_nodes(), dtype=torch.bool)
train_mask[:int(G.number_of_nodes()*0.8)] = 1

val_mask = torch.zeros(G.number_of_nodes(), dtype=torch.bool)
val_mask[int(G.number_of_nodes()*0.8):int(G.number_of_nodes()*0.9)] = 1

test_mask = torch.zeros(G.number_of_nodes(), dtype=torch.bool)
test_mask[int(G.number_of_nodes()*0.9):] = 1

In [13]:
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = tg.nn.GCNConv(num_node_features, 64)
        self.conv2 = tg.nn.GCNConv(64, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.nn.functional.relu(x)
        x = torch.nn.functional.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return torch.nn.functional.log_softmax(x, dim=1)

In [ ]:
# create unsupervised graph convolutional neural network model
class GCN(torch.nn.Module):
    

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_node_features=data.num_nodes, num_classes=NUM_CLASSES).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=5e-4)

In [15]:
for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = torch.nn.functional.nll_loss(out[train_mask], data.y[train_mask])
    loss.backward()
    optimizer.step()

    # validate the model
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = float(pred[val_mask].eq(data.y[val_mask]).sum().item())
    acc = correct / val_mask.sum().item()
    print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}, Val Acc: {acc:.4f}')

Epoch: 001, Loss: 2.3024, Val Acc: 0.0855
Epoch: 002, Loss: 2.2867, Val Acc: 0.0940
Epoch: 003, Loss: 2.2698, Val Acc: 0.1111
Epoch: 004, Loss: 2.2499, Val Acc: 0.1111
Epoch: 005, Loss: 2.2262, Val Acc: 0.0684
Epoch: 006, Loss: 2.1984, Val Acc: 0.0598
Epoch: 007, Loss: 2.1669, Val Acc: 0.0598
Epoch: 008, Loss: 2.1306, Val Acc: 0.0684
Epoch: 009, Loss: 2.0906, Val Acc: 0.0769
Epoch: 010, Loss: 2.0566, Val Acc: 0.0855
Epoch: 011, Loss: 2.0104, Val Acc: 0.1026
Epoch: 012, Loss: 1.9689, Val Acc: 0.0855
Epoch: 013, Loss: 1.9154, Val Acc: 0.0855
Epoch: 014, Loss: 1.8778, Val Acc: 0.0769
Epoch: 015, Loss: 1.8238, Val Acc: 0.0855
Epoch: 016, Loss: 1.7710, Val Acc: 0.0940
Epoch: 017, Loss: 1.7300, Val Acc: 0.0940
Epoch: 018, Loss: 1.6774, Val Acc: 0.1026
Epoch: 019, Loss: 1.6093, Val Acc: 0.1026
Epoch: 020, Loss: 1.5723, Val Acc: 0.1026
Epoch: 021, Loss: 1.5238, Val Acc: 0.1026
Epoch: 022, Loss: 1.4896, Val Acc: 0.1026
Epoch: 023, Loss: 1.4261, Val Acc: 0.1026
Epoch: 024, Loss: 1.3871, Val Acc:

In [16]:
# test the model
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[test_mask].eq(data.y[test_mask]).sum().item())
acc = correct / test_mask.sum().item()
print(f'Test Acc: {acc:.4f}')

Test Acc: 0.0678


In [9]:
adj_matrix = nx.adjacency_matrix(G).todense()
adj_array = np.asarray(adj_matrix)

embedding = SpectralEmbedding(n_components=2)  # 2D embedding
spectral_embedding = embedding.fit_transform(adj_array)

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(spectral_embedding)

/var/folders/h3/9j390gx10pb67vhwk1f_m4sm0000gn/T/ipykernel_63476/2018159809.py:1: FutureWarning:

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.



In [12]:
node2vec = node2vec.Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)

model = node2vec.fit(window=10, min_count=1, batch_words=4)

embeddings = model.wv

kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(embeddings)

Computing transition probabilities:   0%|          | 0/1174 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 50/50 [00:02<00:00, 19.30it/s]
